### Выполнила Зыкова Вероника, БКЛ-192

##### Импорт всех необходиимых модулей

In [1]:
import requests
session = requests.session()
import bs4
import re
import json
import re
import time
from fake_useragent import UserAgent
from random import randint

In [2]:
import pymorphy2
from collections import Counter
from nltk.tokenize import word_tokenize
morph = pymorphy2.MorphAnalyzer()
from random import shuffle

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vzyko\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

##### Выгрузка с irecommend отзывов, имеющих отценки 1 и 5 (отрицательные и положительные соответственно) 

Ссылки, с которых выгружаются данные (их три, потому что у одного больше отрицательнх, у двух других - положительных), и названия файлов

In [4]:
URL = ['https://irecommend.ru/content/anex-tour/', 'https://irecommend.ru/content/ics-travel-group/', \
    'https://irecommend.ru/content/turoperator-kareliya-gid/', 'https://irecommend.ru/content/bolshoi-altai-turoperator/']
BASEURL = 'https://irecommend.ru'
name_neg = 'neg_revs.jsonl'
name_pos = 'pos_revs.jsonl'

Делаем вид, что это браузер, а не питоновский скрипт стучится на сайт

In [23]:
def get_headers():
    ua = UserAgent(verify_ssl=False)
    headers = {'User-Agent': ua.random}
    return headers

Получает оценку отзыва, чтобы открывать ссылки только с нужными и не нервировать защиту сайта

In [24]:
def get_mark(li):
    data = li.find('div', {'class': 'starsRating'}).find_all('div', {'class': 'star'})
    data = [i.find('div').get('class')[0] for i in data].count('on')
    return data

Получает все нужные ссылки со страницы: ссылки на отзывы + ссылку на следующую страницу, если она есть

In [25]:
def get_links(url, marks=(1, 5), headers=None):
    links = {mark: [] for mark in marks}
    response = session.get(url, headers=headers)
    bs_obj = bs4.BeautifulSoup(response.text, features='html.parser')
    data = bs_obj.find('ul', {'class': 'list-comments'}).find_all('li')
    for li in data:
        mark = get_mark(li)
        if int(mark) in marks:
            link = li.find('div', {'class': 'reviewTitle'}).find('a').get('href')
            links[mark].append(link)

    next_link = bs_obj.find('li', {'class': 'pager-item'})
    if next_link:
        next_link = next_link.find('a').get('href')
    else:
        next_link = bs_obj.find('li', {'class': 'pager-last'})
        if next_link:
              next_link = next_link.find('a').get('href')
        else:
              next_link = ''
    return links, next_link

Удаляет из текста все тэги разметки

In [26]:
def delete_tags(line):
    pattern = re.compile(r'<.+?>')
    clean_line = re.sub(pattern, '', line)
    pattern = re.compile(r'\S')
    if not re.match(pattern, clean_line):
        return ''
    else:
        return clean_line.strip()

Получает текст отзыва по ссылке

In [27]:
def get_review(url, headers, base_url=BASEURL):
    url = base_url + url
    response = session.get(url, headers=headers)
    bs_obj = bs4.BeautifulSoup(response.text, features='html.parser')
    summary = delete_tags(str(bs_obj.find('a', {'class': 'review-summary'})))
    text_new = bs_obj.find('div', {'class': 'reviewText'}).find_all('p')
    text = summary
    for el in text_new:
        l = delete_tags(str(el))
        text += ' ' + l
    return text

Собирает все отзывы со страницы

In [48]:
def get_reviews(url, marks=(1, 5)):
    reviews = {}
    headers = get_headers()
    links, next_link = get_links(url, marks=marks, headers=headers)
    print('link done')
    time.sleep(randint(10, 15))
    for mark in links:
        if mark not in reviews:
            reviews[mark] = []
        for link in links[mark]:
            headers = get_headers()
            review = get_review(link, headers=headers)
            print('rev done')
            time.sleep(randint(10, 15))
            reviews[mark].append(review)
    return reviews, next_link

Сохраняет собранное в json, чтобы иметь подстраховку на случай падения программы и просто не скачивать каждый раз заново

In [29]:
def save_jsonl(f_obj_neg, f_obj_pos, data):
    for mark in data:
        line = json.dumps(data[mark])
        if mark >= 3:
            f_obj_pos.write(line + '\n')
        else:
            f_obj_neg.write(line + '\n')

Собирает все вышеперечисленное в одну функцию

In [36]:
def get_all(url, number=200, marks=(1, 5), f_obj_neg=None, f_obj_pos=None):
    reviews = {mark: [] for mark in marks}
    amount = number // 2 + 1
    stop = False
    while not stop:
        stop = True
        new_revs, new_link = get_reviews(url)
        if f_obj_neg and f_obj_pos:
            save_jsonl(f_obj_neg, f_obj_pos, new_revs)
        for mark in marks:
            reviews[mark].extend(new_revs[mark])
            if len(reviews[mark]) < amount:
                stop = False
            print(f'{mark}: done {len(reviews[mark])} of {amount}')
        if new_link != '':
            url = BASEURL + new_link
        else:
            stop = True
    return reviews

Вызывает основную функцию со всеми нужными параметрами

In [11]:
with open(name_neg, 'a', encoding='utf-8') as f_neg, open(name_pos, 'a', encoding='utf-8') as f_pos:
    reviews = get_all(URL[3], number=200, f_obj_neg=f_neg, f_obj_pos=f_pos, marks=(5, ))

##### Предобработка данных

Читает файл с данными, где каждыя строка - json

In [5]:
def read_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.read()
    data = data.splitlines()
    texts = []
    for el in data:
        texts.extend(json.loads(el))
    return texts

Токенизация, очистка от пунктуации и прочих не-слов, лемматизация

In [6]:
def clean_text(text):
    tokens = word_tokenize(text)
    lemmas = []
    for token in tokens:
        if token.isalpha():
            token_pr = morph.parse(token.lower())[0] 
            norm_form = token_pr.normal_form
            lemmas.append(norm_form)
    return lemmas

Применяет предыдущую функцию ко всем текстам

In [7]:
def clean_all(reviews):
    clean_reviews = {}
    for mark in reviews:
        clean_reviews[mark] = []
        for el in reviews[mark]:
            text = clean_text(el)
            clean_reviews[mark].extend(text)
    return clean_reviews

Разделяет все тексты на тестовую и обучающую выборки, при этом уравнивая количество, если надо

In [8]:
def train_test(revs, perc=0.2, same=True):
    train = {}
    test = {}
    if same:
        line = min([len(revs[i]) for i in revs])
    for el in revs:
        if not same:
            line = len(revs[el])
        shuffle(revs[el])
        test[el] = revs[el][0:int(perc * line)]
        train[el] = revs[el][int(perc * line):line]
    return train, test

In [9]:
def get_amount(revs):
    res = {mark: len(revs[mark]) for mark in revs}
    return res

Читаем тексты, очищаем обучающую часть

In [10]:
revs = {}
revs['neg'] = read_file(name_neg)
revs['pos'] = read_file(name_pos)
train_text, test_text = train_test(revs)
train = clean_all(train_text)

Создаем множества, не включая в них слова с частотой меньше нужной

In [11]:
def get_sets(revs, freq_line=10):
    neg_set_full = set(revs['neg'])
    pos_set_full = set(revs['pos'])
    neg_freq = Counter(revs['neg'])
    pos_freq = Counter(revs['pos'])
    neg_set = neg_set_full - pos_set_full
    pos_set = pos_set_full - neg_set_full
    neg_set_tot = set([el for el in neg_set if neg_freq[el] >= freq_line])
    pos_set_tot = set([el for el in pos_set if pos_freq[el] >= freq_line])
    return neg_set_tot, pos_set_tot

##### Проверка

Получаем количество слов, относящихся к негативному и позитивному классу соответственно

In [12]:
def get_score(text, neg, pos):
    text = clean_text(text)
    score = [0, 0]
    for el in text:
        if el in neg:
            score[0] += 1
        elif el in pos:
            score[1] += 1
    return tuple(score)

Получаем тип отзыва на основе результата предыдущей функции

In [13]:
def get_type(text, neg, pos):
    score = get_score(text, neg, pos)
    return score[1] > score[0]

Приводим выборку к нужному нам виду

In [14]:
def get_test(test):
    test_all = []
    marks = {'neg': 0, 'pos': 1}
    for mark in test:
        for el in test[mark]:
            test_all.append([el, marks[mark]])
    return test_all

Оцениваем определение тональности всех отзывов в выборке

In [15]:
def evaluate(data, neg, pos):
    score = 0
    for el in data: 
        text_type = get_type(el[0], neg, pos)
        if text_type == el[1]:
            score += 1
    return score / len(data)

Собираем множества, граница частотности указана в соотвествии с результатом перебора ниже

In [21]:
neg, pos = get_sets(train, freq_line=3)

In [22]:
test = get_test(test_text)
train_new = get_test(train_text)

Лучший из полученных результатов

In [23]:
evaluate(train_new, neg, pos) # на обучающей

0.9833333333333333

In [24]:
evaluate(test, neg, pos) # на тестовой

0.8666666666666667

Перебираем параметры частотности

In [20]:
for i in range(10):
    neg, pos = get_sets(train, freq_line=i)
    print(f'{i}: {evaluate(test, neg, pos)}')

0: 0.8666666666666667
1: 0.8666666666666667
2: 0.8333333333333334
3: 0.8666666666666667
4: 0.8333333333333334
5: 0.7333333333333333
6: 0.7666666666666667
7: 0.7666666666666667
8: 0.7
9: 0.6666666666666666


##### Улучшения

1. Можно подсчитывать не просто количество слов из положительного или отрицательного списка, а использовать еще и то, как эти слова распределены по частоте в отзывах. Например, если слово изста положительных отзывов встретилось только в одном, а в отрицательных не встретилось вообще, то достаточно странно считать его маркером положительного отзыва, так как у нас просто слишком мало данных, чтобы что-то такое утверждать. С другой стороны, слово ужасно однозначно встречается во многих отрицательных отзывах и является более достоверным маркером. Чтобы различать такие ситуации, можно при подсчете баллов за положительный или отрицательный ответ прибавлять не единицу, а долю отзывов, содержащих это слово.

2. Также можно перейти на вероятностную модель ответа (с помощью описанного выше это будет не так сложно): для каждого отзыва считать не какой он по тональности, а с какой вероятностью он имеет какую-то тональность. Таким образом можно будет задавать границу для определения отношения к классу: если нам вдргу захочется, чтобы все отзывы, отмеченные негативом, точно такими были, то мы можем задать высокий порог вероятности для негативных отзывов.